# Part B: Task 1

**(a)** <br>
Import libraries

In [1]:
# import needed library
import json
import pandas as pd
from pprint import pprint
from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient

#kafka required libraries
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt

Since the question want us to find the latest date in climate data that was inserted into the MongoDB database i had created in Part A, Task 2, so i will connect to the database first

In [2]:
# Method 1: specify the host and port explicitly
client = MongoClient('192.168.1.27', 27017) # 27017 number of ports


In [3]:
# to get all the databases in client 
result = client.list_database_names()
print(result)

['admin', 'config', 'fit3182_assignment_db', 'fit3182_db', 'local']


In [4]:
# connect to database
db = client.fit3182_assignment_db

In [5]:
# list collections
print(db.list_collection_names())

['climate_hotspot', 'hotspot', 'climate', 'clihot']


In [6]:
# connect to the collection 
cli_hot = db.clihot

In [7]:
#find the latest climate date

data = cli_hot.aggregate([{"$sort": {"date": 1}},{"$limit": 1}, {"$project": {"date": {"$dateToString": {"format": "%Y-%m-%d", "date": "$date"}}}}])
for res in data:
    pprint(res)

{'_id': ObjectId('647f53babeaf2b433c177939'), 'date': '2021-12-31'}


so we can see from the result the latest date would be 31/12/2021

read the data using pandas

In [8]:
climatestream = pd.read_csv("./climate_streaming.csv")

**Data inspection**<br>
inspect on the data and see how the data looks like, what fields it contains

In [9]:
climatestream.head()

,latitude,longitude,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2
0,-37.623,149.323,19,56.8,7.9,11.1,0.00I,154
1,-38.038,142.986,15,50.7,9.2,13.0,0.02G,128
2,-37.950,142.366,16,53.6,8.1,15.0,0.00G,133
3,-38.231,147.172,24,61.6,7.7,14.0,0.00I,186
4,-37.903,145.250,24,62.3,7.0,13.0,0.00I,185


**Json format**<br>
convert the csv data into json format

In [10]:
clistreamjs = climatestream.to_json(orient='records')
clistreamjs = json.loads(clistreamjs)

In [11]:
pprint(clistreamjs)

[{'GHI_w/m2': 154,
  'air_temperature_celcius': 19,
  'latitude': -37.623,
  'longitude': 149.323,
  'max_wind_speed': 11.1,
  'precipitation ': ' 0.00I',
  'relative_humidity': 56.8,
  'windspeed_knots': 7.9},
 {'GHI_w/m2': 128,
  'air_temperature_celcius': 15,
  'latitude': -38.038,
  'longitude': 142.986,
  'max_wind_speed': 13.0,
  'precipitation ': ' 0.02G',
  'relative_humidity': 50.7,
  'windspeed_knots': 9.2},
 {'GHI_w/m2': 133,
  'air_temperature_celcius': 16,
  'latitude': -37.95,
  'longitude': 142.366,
  'max_wind_speed': 15.0,
  'precipitation ': ' 0.00G',
  'relative_humidity': 53.6,
  'windspeed_knots': 8.1},
 {'GHI_w/m2': 186,
  'air_temperature_celcius': 24,
  'latitude': -38.231,
  'longitude': 147.172,
  'max_wind_speed': 14.0,
  'precipitation ': ' 0.00I',
  'relative_humidity': 61.6,
  'windspeed_knots': 7.7},
 {'GHI_w/m2': 185,
  'air_temperature_celcius': 24,
  'latitude': -37.903,
  'longitude': 145.25,
  'max_wind_speed': 13.0,
  'precipitation ': ' 0.00I',
  '

# Kafka producer 

**To publish the message**

**IP address**

In [12]:
host_ip = "192.168.1.27"

**Function to publish the message**

In [15]:
def publish_message(producer_instance, topic_name, key, data):
    try:
        # encode the date into bytes
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))



**connect to kafka producer**

In [14]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{host_ip}:9092'],
                                  # kafka only accept bytes
                                  # convert message to bytes before sending
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer



**basically to run the message publishing**

In [16]:
if __name__ == '__main__':
   
    topic = 'climate'
    
    print('Publishing records..')
    producer01 = connect_kafka_producer()
    
    current_datetime = datetime.now()
    new_date = datetime.strptime("2021-12-31", "%Y-%m-%d")
    
    while True:
        data = random.choice(clistreamjs)
        
        if datetime.now() - current_datetime >= timedelta(seconds=10):
            new_date = new_date+timedelta(days=1)
        # identified from previous the lastest date would be 31/12/2021
        data["producer"] = "climate_streaming"
        data["newdate"] = str(new_date)
        
        # add in created date time for climate data as it is needed for data visualization
        data['cre_date_time'] = str(dt.datetime.now().strftime("%X"))
        print(data)
        publish_message(producer01, topic, 'jsondata', data)
        current_datetime = datetime.now()
        sleep(10)

Publishing records..
{'latitude': -37.293, 'longitude': 141.245, 'air_temperature_celcius': 10, 'relative_humidity': 45.7, 'windspeed_knots': 3.7, 'max_wind_speed': 8.0, 'precipitation ': ' 0.01G', 'GHI_w/m2': 89, 'producer': 'climate_streaming', 'newdate': '2021-12-31 00:00:00', 'cre_date_time': '16:02:46'}
Message published successfully. Data: {'latitude': -37.293, 'longitude': 141.245, 'air_temperature_celcius': 10, 'relative_humidity': 45.7, 'windspeed_knots': 3.7, 'max_wind_speed': 8.0, 'precipitation ': ' 0.01G', 'GHI_w/m2': 89, 'producer': 'climate_streaming', 'newdate': '2021-12-31 00:00:00', 'cre_date_time': '16:02:46'}
{'latitude': -35.321, 'longitude': 143.502, 'air_temperature_celcius': 12, 'relative_humidity': 44.0, 'windspeed_knots': 4.7, 'max_wind_speed': 7.0, 'precipitation ': ' 0.00G', 'GHI_w/m2': 108, 'producer': 'climate_streaming', 'newdate': '2022-01-01 00:00:00', 'cre_date_time': '16:02:56'}
Message published successfully. Data: {'latitude': -35.321, 'longitude': 

KeyboardInterrupt: 